In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from collections import defaultdict
from joblib import Parallel, delayed

import sqlite3
import sys
import time
import math
#import tqdm
from tqdm.auto import tqdm
import datetime
import os
import pickle
from pathlib import Path

from glicko2 import Player
import multiprocessing

tqdm.pandas()

if os.path.exists('/workspace/data'):
    # Load the dictionary of DataFrames from the pickle
    data_path = '/workspace/data/'
else:
    data_path = '../data/'

## Loading SQLite Database into Pandas DataFrames

The following code connects to an SQLite database (`melee_player_database.db`) and converts each table within the database into a pandas DataFrame. The DataFrames will be stored in a dictionary, where each key corresponds to the table name with `_df` appended, and the values are the respective DataFrames.

### Steps:

1. **Database Connection**: We use the `sqlite3` library to connect to the SQLite database file.
2. **Retrieve Table Names**: A query retrieves all the table names in the database.
3. **Convert Tables to DataFrames**: For each table:
   - The table is loaded into a pandas DataFrame using `pd.read_sql()`.
   - We check each column to see if any data is JSON-formatted (lists or dictionaries). If so, we convert these columns from strings into their corresponding Python objects using `json.loads()`.
4. **Store DataFrames**: The DataFrames are stored in a dictionary, where the key is the table name with a `_df` suffix, and the value is the DataFrame.
5. **Database Connection Closed**: Once all tables are loaded into DataFrames, the database connection is closed.

### Example:
If the database contains a table named `players`, the corresponding DataFrame will be stored in the dictionary with the key `players_df`, and can be accessed as:

```python
players_df = dfs['players_df']


In [ ]:
# Function to get the table names
def get_table_names(conn):
    query = "SELECT name FROM sqlite_master WHERE type='table';"
    return pd.read_sql(query, conn)['name'].tolist()

# Function to load tables into DataFrames
def load_tables_to_dfs(conn):
    table_names = get_table_names(conn)
    dataframes = {}
    
    for table in table_names:
        # Load table into a DataFrame
        df = pd.read_sql(f"SELECT * FROM {table}", conn)
        
        # Detect and convert JSON formatted columns (if any)
        for col in df.columns:
            # Check if any entry in the column is a valid JSON (list or dictionary)
            if df[col].apply(lambda x: isinstance(x, str)).all():
                try:
                    # Try parsing the column as JSON
                    df[col] = df[col].apply(lambda x: json.loads(x) if pd.notnull(x) else x)
                except (json.JSONDecodeError, TypeError):
                    # If it fails, skip the column
                    pass
        
        # Store the DataFrame with table name + '_df'
        dataframes[f"{table}_df"] = df
        
    return dataframes

if os.path.exists(data_path + 'dfs_dict.pkl'):
    cell_has_run = True
    # Load the dictionary of DataFrames from the pickle
    with open(data_path + 'dfs_dict.pkl', 'rb') as f:
        dfs = pickle.load(f)
# Check if the flag variable exists in the global scope so that this code does not run twice
if 'cell_has_run' not in globals():
    path = data_path + "melee_player_database.db"
    
    # Connect to the database
    conn = sqlite3.connect(path)

    # Convert each table into a DataFrame
    dfs = load_tables_to_dfs(conn)

    # Close the connection
    conn.close()

    # Now, you have a dictionary 'dfs' where each key is the table name with '_df' suffix and value is the corresponding DataFrame.
    # For example, to access the DataFrame for a table called 'players':
    # players_df = dfs['players_df']

    dfs['tournament_info_df']['start'] = pd.to_datetime(dfs['tournament_info_df']['start'], unit='s')
    dfs['tournament_info_df']['end'] = pd.to_datetime(dfs['tournament_info_df']['end'], unit='s')

    
    # Set the flag to indicate that the cell has been run
    cell_has_run = True

### Here we adjust the data types of the dataframes so that they are the correct type. (This will be updated as needed.)

In [ ]:
dfs['sets_df']['best_of'] = dfs['sets_df']['best_of'].fillna(0).astype(int) 

### Here we make dataframes that we will use and print the head.

The integers in 'characters' count the number of games the player has played that character. (We verify this for Zain below.)

In [ ]:
players_df = dfs['players_df']
players_df.head()

In [ ]:
ranking_df = dfs['ranking_df']
ranking_df.head()

In [ ]:
ranking_seasons_df = dfs['ranking_seasons_df']
ranking_seasons_df.head()

In [ ]:
sets_df = dfs['sets_df']
print(f"{sets_df[sets_df['game_data'].apply(lambda x: len(x) > 0)].shape[0] / sets_df.shape[0]:0.01%} percent of sets have some game data)")

sets_df.head()

In [ ]:
tournament_info_df = dfs['tournament_info_df']
tournament_info_df.head()

In [ ]:
# Code optimization by Dan
# Basically we want to replace this line in process_tournament with something more efficient:
#
#      tournament_sets_df = sets_df[sets_df['tournament_key'] == tournament_key]
#
# Instead, we can
# - Merge the tournament date info into ``sets_df``
# - Sort by date
# - Store the start/end positions of each tournament in a separate dictionary
# - Use tournament_sets_df = sets_df.iloc[start:end+1] instead.

sets_df = sets_df.merge(tournament_info_df[['key', 'start', 'end']], left_on='tournament_key', right_on='key', how='left')
sets_df = sets_df.drop(labels=['key_y'], axis='columns')
sets_df = sets_df.rename(columns={"key_x": "key"})
sets_df = sets_df.sort_values(by=['end', 'tournament_key']) # Just in case there are tournaments with the exact same end date

In [ ]:
# A bit of data cleanup
# TODO: Rerun!
min_date = datetime.datetime(2015, 1, 1)
max_date = datetime.datetime(2024, 12, 31)

sets_df = sets_df[(sets_df['start'] >= min_date) & (sets_df['end'] >= min_date) & (sets_df['start'] <= max_date) & (sets_df['end'] <= max_date)]

In [ ]:
# Example of game data. List of dictionaries.
sets_df[sets_df['game_data'].apply(lambda x: x != [])].iloc[0]['game_data']

## Another variation

This instead adds the character used to the player id (ex: 1021/yoshi) and treats them as separate entities. It then just computes the elo for each of them.

In [ ]:
# Assumes dataset_generation/game_data_extractor.ipynb was run
game_data_df = pd.read_pickle(data_path + 'individual_game_data.pkl')
game_data_df

In [ ]:
game_data_df['p1_id'] = game_data_df['p1_id'] + '/' + game_data_df['p1_char']
game_data_df['p2_id'] = game_data_df['p2_id'] + '/' + game_data_df['p2_char']

game_data_df['winner_id'] = game_data_df['winner_id'] + '/' + game_data_df['winner_char']
game_data_df['loser_id'] = game_data_df['loser_id'] + '/' + game_data_df['loser_char']

game_data_df.drop(['p1_char', 'p2_char', 'winner_char', 'loser_char'], axis=1, inplace=True)
game_data_df

In [ ]:
# Just to make sure, but it should already be sorted
game_data_df = game_data_df.sort_values(by='end')
game_data_df

## Compute the rankings

In [ ]:
# TLDR the ELO computation occasionally seems to go negative, and I think this can cause a lot of problems.
# Likewise, the RD value seems to get unusually large occasionally
MIN_ELO = 500.0
MAX_RD = 350.0

def process_tournament(player_ratings_df, player_rds_df, sets_df, positions, tournament_key, key_is_date, glicko_objects):
    # Get the games for this specific period, using precomputed indices for speed.
    period_df = sets_df.iloc[positions[tournament_key][0]:positions[tournament_key][1]+1]
    unique_players = list(player_ratings_df.columns)

    # Prepare player matches DataFrame.
    # Essentially, one copy where p1 is the player of consideration,
    # and then another copy where p2 is under consideration.
    df_p1 = period_df[['p1_id', 'p2_id', 'winner_id']].copy()
    df_p1.rename(columns={'p1_id': 'player_id', 'p2_id': 'opponent_id'}, inplace=True)
    df_p1['outcome'] = (df_p1['winner_id'] == df_p1['player_id']).astype(int)
    df_p1 = df_p1[['player_id', 'opponent_id', 'outcome']]
    
    df_p2 = period_df[['p2_id', 'p1_id', 'winner_id']].copy()
    df_p2.rename(columns={'p2_id': 'player_id', 'p1_id': 'opponent_id'}, inplace=True)
    df_p2['outcome'] = (df_p2['winner_id'] == df_p2['player_id']).astype(int)
    df_p2 = df_p2[['player_id', 'opponent_id', 'outcome']]

    outcomes_df = pd.concat([df_p1, df_p2], ignore_index=True)

    # Add opponent ELO and RD values
    outcomes_df['opponent_rating'] = outcomes_df['opponent_id'].apply(lambda x: glicko_objects[x].getRating())
    outcomes_df['opponent_rd'] = outcomes_df['opponent_id'].apply(lambda x: glicko_objects[x].getRd())

    # Compile the opponent data into lists (one for ratings, one for rds, one for outcomes) for every player
    grouped = outcomes_df.groupby('player_id').agg({
        'opponent_rating': list,
        'opponent_rd': list,
        'outcome': list
    }).reset_index()

    # We will need to update the glicko objects separately for players that did or didn't play in this rating period
    players_with_games = list(grouped['player_id'])
    players_without_games = [x for x in unique_players if x not in players_with_games]

    #First, let's handle the players that did play games in this period
    def update_glicko(row):
        glicko_objects[row['player_id']].update_player(row['opponent_rating'], row['opponent_rd'], row['outcome'])

    grouped.apply(update_glicko, axis=1)

    # Now let's handle players that did NOT play games in this period
    for player in players_without_games:
        glicko_objects[player].did_not_compete()

    # This might be a bug in the glicko2 library that we are using,
    # but occasionally we get weird values for the rating and/or RD
    # and this DOES very occasionally seem to cause the whole thing to crash
    # (division by zero somewhere)
    # and so we use professional-grade duct tape here
    for player in unique_players:
        if glicko_objects[player].getRating() < MIN_ELO:
            glicko_objects[player].setRating(MIN_ELO)

        if glicko_objects[player].getRd() > MAX_RD:
            glicko_objects[player].setRd(MAX_RD)

    # Now update that specific row of the player ratings dataframe with the new glicko data
    date = tournament_key if key_is_date else sets_df.iloc[positions[tournament_key][0]]['end']

    #new_player_data = {}
    #for player in unique_players:
    #    new_player_data[player] = glicko_objects[player].getRating()

    # These should be in the same order as the columns, as unique_players was used to create the columns as well.
    # This is slightly more efficient than using a dictionary.
    new_player_ratings = [glicko_objects[player].getRating() for player in unique_players]
    new_player_rds     = [glicko_objects[player].getRd()     for player in unique_players]

    player_ratings_df.loc[date] = new_player_ratings
    player_rds_df.loc[date]     = new_player_rds

In [ ]:
date_sets_positions = {}

initial_date = datetime.datetime(2015, 1, 1)
interval = datetime.timedelta(weeks=1)

end_date = game_data_df['end'].max()

date = initial_date

# Loop through and test for containment in the interval [date, date+interval)
while date + interval <= end_date:
    is_in_interval = (game_data_df['end'] >= date) & (game_data_df['end'] < date + interval)
    is_in_interval = is_in_interval.reset_index(drop=True) # Make absolutely sure it can be used for iloc

    if (~is_in_interval).all(): # all False
        date_sets_positions[date + interval] = (0,-1)
    else: # At least one True
        date_sets_positions[date + interval] = (is_in_interval.idxmax(), is_in_interval[::-1].idxmax())

    date += interval

date_sets_positions

In [ ]:
unique_players = list(set(list(game_data_df['p1_id'].unique()) + list(game_data_df['p2_id'].unique())))

# Note that this technically starts at initial_date + interval, not initial_date
dates = [x for x in date_sets_positions]

# Convenient store of glicko objects
glicko_objects = {}
for player in unique_players:
    glicko_objects[player] = Player()

# Initial row of default ELO values, at the earliest possible date.
# Note that we are also pre-allocating the dataframe with the rest of the possible dates.
player_ratings_df = pd.DataFrame([[1500.0] * len(unique_players)], columns=unique_players, index=[initial_date] + dates)
player_rds_df = pd.DataFrame([[350.0] * len(unique_players)], columns=unique_players, index=[initial_date] + dates)

# Loop over the individual date intervals above.
for specific_date in tqdm(dates, total=len(dates)):
    process_tournament(player_ratings_df, player_rds_df, game_data_df, date_sets_positions, specific_date, True, glicko_objects)

In [ ]:
player_ratings_df

## Save data

In [ ]:
player_ratings_df.to_pickle(data_path + 'player_char_overall_rankings_weekly.pkl')
player_rds_df.to_pickle(data_path + 'player_char_overall_rds_weekly.pkl')